In [1]:
import pandas as pd, numpy as np

In [2]:
from vivarium import Artifact
from get_draws.api import get_draws

In [3]:
art = Artifact('/ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf',
               filter_terms=['year_start==2019', f'age_start>=0.5'])

# Define constants

## Artifact values

In [4]:
wasting_exposure = art.load('risk_factor.child_wasting.exposure').reset_index()
wasting_paf = art.load('risk_factor.child_wasting.population_attributable_fraction').reset_index().drop(columns='affected_measure')
wasting_rr = art.load('risk_factor.child_wasting.relative_risk').reset_index()
wasting_rr_dd = wasting_rr.loc[wasting_rr.affected_entity=='diarrheal_diseases'].drop(columns=['affected_entity','affected_measure'])
wasting_rr_lri = wasting_rr.loc[wasting_rr.affected_entity=='lower_respiratory_infections'].drop(columns=['affected_entity','affected_measure'])
wasting_rr_measles = wasting_rr.loc[wasting_rr.affected_entity=='measles'].drop(columns=['affected_entity','affected_measure'])

prevalence_c302 = art.load('cause.diarrheal_diseases.prevalence') 
p_1 = wasting_exposure.loc[wasting_exposure.parameter=='cat1'].set_index([c for c in wasting_exposure.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
p_2 = wasting_exposure.loc[wasting_exposure.parameter=='cat2'].set_index([c for c in wasting_exposure.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
p_3 = wasting_exposure.loc[wasting_exposure.parameter=='cat3'].set_index([c for c in wasting_exposure.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
p_4 = wasting_exposure.loc[wasting_exposure.parameter=='cat4'].set_index([c for c in wasting_exposure.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')

ACMR = art.load('cause.all_causes.cause_specific_mortality_rate') 
csmr_c302 = art.load('cause.diarrheal_diseases.cause_specific_mortality_rate')
emr_c302 = art.load('cause.diarrheal_diseases.excess_mortality_rate')
csmr_pem = art.load('cause.protein_energy_malnutrition.cause_specific_mortality_rate')
emr_pem = art.load('cause.protein_energy_malnutrition.excess_mortality_rate')
csmr_lri = art.load('cause.lower_respiratory_infections.cause_specific_mortality_rate')
csmr_measles = art.load('cause.measles.cause_specific_mortality_rate')
paf_wasting_lri = wasting_paf.loc[wasting_paf.affected_entity=='lower_respiratory_infections'].set_index([c for c in wasting_paf.columns if 'draw' not in c and c != 'affected_entity']).drop(columns='affected_entity')
paf_wasting_measles = wasting_paf.loc[wasting_paf.affected_entity=='measles'].set_index([c for c in wasting_paf.columns if 'draw' not in c and c != 'affected_entity']).drop(columns='affected_entity')
paf_wasting_c302 = wasting_paf.loc[wasting_paf.affected_entity=='diarrheal_diseases'].set_index([c for c in wasting_paf.columns if 'draw' not in c and c != 'affected_entity']).drop(columns='affected_entity')
incidence_c302 = art.load('cause.diarrheal_diseases.incidence_rate') 
remission_c302 = art.load('cause.diarrheal_diseases.remission_rate') 
RR_lri_1 = wasting_rr_lri.loc[wasting_rr_lri.parameter=='cat1'].set_index([c for c in wasting_rr_lri.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_lri_2 = wasting_rr_lri.loc[wasting_rr_lri.parameter=='cat2'].set_index([c for c in wasting_rr_lri.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_lri_3 = wasting_rr_lri.loc[wasting_rr_lri.parameter=='cat3'].set_index([c for c in wasting_rr_lri.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_measles_1 = wasting_rr_measles.loc[wasting_rr_measles.parameter=='cat1'].set_index([c for c in wasting_rr_measles.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_measles_2 = wasting_rr_measles.loc[wasting_rr_measles.parameter=='cat2'].set_index([c for c in wasting_rr_measles.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_measles_3 = wasting_rr_measles.loc[wasting_rr_measles.parameter=='cat3'].set_index([c for c in wasting_rr_measles.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_1 = wasting_rr_dd.loc[wasting_rr_dd.parameter=='cat1'].set_index([c for c in wasting_rr_dd.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_2 = wasting_rr_dd.loc[wasting_rr_dd.parameter=='cat2'].set_index([c for c in wasting_rr_dd.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_3 = wasting_rr_dd.loc[wasting_rr_dd.parameter=='cat3'].set_index([c for c in wasting_rr_dd.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')
RR_4 = wasting_rr_dd.loc[wasting_rr_dd.parameter=='cat4'].set_index([c for c in wasting_rr_dd.columns if 'draw' not in c and c != 'parameter']).drop(columns='parameter')

In [5]:
# calculate custom prevalence to be consistent
prevalence_c302 = incidence_c302 * (4.3/365)

In [6]:
remission_c302 = 1/(4.3*365) * prevalence_c302
remission_c302

draw_0    draw_1    draw_2  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      0.000021  0.000022  0.000022   
       1.0       2.0     2019       2020      0.000011  0.000012  0.000014   
       2.0       5.0     2019       2020      0.000011  0.000012  0.000014   
Male   0.5       1.0     2019       2020      0.000021  0.000021  0.000022   
       1.0       2.0     2019       2020      0.000011  0.000013  0.000014   
       2.0       5.0     2019       2020      0.000011  0.000013  0.000014   

                                                draw_3    draw_4    draw_5  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      0.000023  0.000023  0.000019   
       1.0       2.0     2019       2020      0.000012  0.000015  0.000012   
       2.0       5.0     2019       2020      0.000012  0.000015  0.000012   
Male   0.5       1.0     2019       2020      0.000023  0.000023  0.000019   
       1.0       2.0     2019       2020      0.000011  0.000015  0.000012   
       2.0       5.0     2019       2020      0.000011  0.000015  0.000012   

                                                draw_6    draw_7    draw_8  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      0.000019  0.000023  0.000025   
       1.0       2.0     2019       2020      0.000011  0.000015  0.000012   
       2.0       5.0     2019       2020      0.000011  0.000015  0.000012   
Male   0.5       1.0     2019       2020      0.000019  0.000023  0.000026   
       1.0       2.0     2019       2020      0.000011  0.000014  0.000012   
       2.0       5.0     2019       2020      0.000011  0.000014  0.000012   

                                                draw_9  ...  draw_990  \
sex    age_start age_end year_start year_end            ...             
Female 0.5       1.0     2019       2020      0.000021  ...  0.000020   
       1.0       2.0     2019       2020      0.000012  ...  0.000014   
       2.0       5.0     2019       2020      0.000012  ...  0.000014   
Male   0.5       1.0     2019       2020      0.000021  ...  0.000020   
       1.0       2.0     2019       2020      0.000012  ...  0.000015   
       2.0       5.0     2019       2020      0.000012  ...  0.000015   

                                              draw_991  draw_992  draw_993  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      0.000018  0.000020  0.000022   
       1.0       2.0     2019       2020      0.000009  0.000014  0.000013   
       2.0       5.0     2019       2020      0.000009  0.000014  0.000013   
Male   0.5       1.0     2019       2020      0.000018  0.000020  0.000022   
       1.0       2.0     2019       2020      0.000009  0.000013  0.000013   
       2.0       5.0     2019       2020      0.000009  0.000013  0.000013   

                                              draw_994  draw_995  draw_996  \
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      0.000019  0.000020  0.000027   
       1.0       2.0     2019       2020      0.000011  0.000010  0.000015   
       2.0       5.0     2019       2020      0.000011  0.000010  0.000015   
Male   0.5       1.0     2019       2020      0.000019  0.000020  0.000027   
       1.0       2.0     2019       2020      0.000011  0.000011  0.000015   
       2.0       5.0     2019       2020      0.000011  0.000011  0.000015   

                                              draw_997  draw_998  draw_999  
sex    age_start age_end year_start year_end                                
Female 0.5       1.0     2019       2020      0.000022  0.000018  0.000020  
       1.0       2.0     2019       2020      0.000011  0.000012  0.000011  
       2.0       5.0     2019 

In [28]:
emr_c302 = csmr_c302 / prevalence_c302

## Wasting transition rates

These were read-in from simulation model output rather than calculating them directly given the complexity of the calculations. This should be updated as the model is finalized. The exception is for r4 and i3 (which are currently undergoing calibration)


In [29]:
output_dir ='/ihme/costeffectiveness/results/vivarium_ciff_sam/v5.3.2_remission_rate_changes/ciff_sam/2022_01_28_17_10_48/count_data/'
wasting_transitions = pd.read_csv(output_dir + 'wasting_transition_count.csv').drop(columns='Unnamed: 0')
wasting_transitions = (wasting_transitions
                       .loc[wasting_transitions.scenario=='baseline']
                       .groupby(['input_draw','sex','age','measure']).sum()
                       .drop(columns='year')
                       .reset_index())
wasting_person_time = pd.read_csv(output_dir + 'wasting_state_person_time.csv').drop(columns='Unnamed: 0')
wasting_person_time = (wasting_person_time
                       .loc[wasting_person_time.scenario=='baseline']
                       .groupby(['input_draw','sex','age','cause']).sum()
                       .drop(columns='year')
                       .reset_index())

In [30]:
def compute_wasting_transition_rate(transition_count_parameter, person_time_state, ages):
    rate = (((wasting_transitions.loc[wasting_transitions.measure==f'{transition_count_parameter}']
           .set_index(['input_draw','sex','age']).drop(columns='measure'))
          / (wasting_person_time.loc[wasting_person_time.cause==f'{person_time_state}']
             .set_index(['input_draw','sex','age']).drop(columns='cause')))
          .groupby(['sex','age']).mean().reset_index())
    rate['sex'] = rate.sex.str.capitalize()
    rate['year_start']=2019
    rate['year_end']=2020
    rate = rate.loc[rate.age.isin(ages)]
    rate['age_start'] = np.where(rate.age=='6-11_months',0.5,
                                np.where(rate.age=='12_to_23_months', 1.0, 2.0))
    rate['age_end'] = np.where(rate.age=='6-11_months',1.0,
                                np.where(rate.age=='12_to_23_months', 2.0, 5.0))
    for i in list(range(0,1000)):
        rate[f'draw_{i}'] = rate['value']
    return rate.drop(columns=['age','value']).set_index(['sex','age_start','age_end','year_start','year_end']).sort_index()

In [31]:
ages = ['6-11_months','12_to_23_months','2_to_4']
t1 = compute_wasting_transition_rate('severe_acute_malnutrition_to_mild_child_wasting_event_count', 
                                     'severe_acute_malnutrition', ages)
r2 = compute_wasting_transition_rate('severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count', 
                                     'severe_acute_malnutrition', ages)
r3 = compute_wasting_transition_rate('moderate_acute_malnutrition_to_mild_child_wasting_event_count', 
                                     'moderate_acute_malnutrition', ages)
r4_obs = compute_wasting_transition_rate('mild_child_wasting_to_susceptible_to_child_wasting_event_count', 
                                     'mild_child_wasting', ages)
i1 = compute_wasting_transition_rate('moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                     'moderate_acute_malnutrition', ages)
i2 = compute_wasting_transition_rate('mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                     'mild_child_wasting', ages)
i3_obs = compute_wasting_transition_rate('susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                     'susceptible_to_child_wasting', ages)


# why do I overwrite these?

r4_daily = 1/50
r4 = -np.log(1 - r4_daily) * 365
ap0 = 1 - np.exp(-ACMR * 1 / 365)
m_D4_daily = 1 - np.exp(-(ACMR  
        - csmr_c302 + csmr_c302 * (1 - paf_wasting_c302)
        - csmr_lri + csmr_lri * (1 - paf_wasting_lri)
        - csmr_measles + csmr_measles * (1 - paf_wasting_measles)) / 365)

# why is this "D4"? instead of just 4?

i3_daily = ap0 * p_4 / (p_4 / (ap0+1)) + p_3 / (ap0 + 1) * r4_daily / (p_4 / (ap0+1)) - m_D4_daily
i3 = -np.log(1 - i3_daily) * 365

<ipython-input-30-454c7b47a107>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rate[f'draw_{i}'] = rate['value']


In [50]:
ages = ['6-11_months','12_to_23_months','2_to_4']
t1 = compute_wasting_transition_rate('severe_acute_malnutrition_to_mild_child_wasting_event_count', 
                                     'severe_acute_malnutrition', ages)
r2 = compute_wasting_transition_rate('severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count', 
                                     'severe_acute_malnutrition', ages)
r3 = compute_wasting_transition_rate('moderate_acute_malnutrition_to_mild_child_wasting_event_count', 
                                     'moderate_acute_malnutrition', ages)
r4_obs = compute_wasting_transition_rate('mild_child_wasting_to_susceptible_to_child_wasting_event_count', 
                                     'mild_child_wasting', ages)
i1 = compute_wasting_transition_rate('moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                     'moderate_acute_malnutrition', ages)
i2 = compute_wasting_transition_rate('mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                     'mild_child_wasting', ages)
i3_obs = compute_wasting_transition_rate('susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                     'susceptible_to_child_wasting', ages)


r4_daily = 1/50
r4 = -np.log(1 - r4_daily) * 365
ap0 = 1 - np.exp(-ACMR * 1 / 365)
m_D4_daily = 1 - np.exp(-(ACMR  
        - csmr_c302 + csmr_c302 * (1 - paf_wasting_c302)
        - csmr_lri + csmr_lri * (1 - paf_wasting_lri)
        - csmr_measles + csmr_measles * (1 - paf_wasting_measles)) / 365)

i3_daily = ap0 * p_4 / (p_4 / (ap0+1)) + p_3 / (ap0 + 1) * r4_daily / (p_4 / (ap0+1)) - m_D4_daily
i3 = -np.log(1 - i3_daily) * 365

<ipython-input-30-454c7b47a107>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rate[f'draw_{i}'] = rate['value']


## Prevalence ratios

In [51]:
PR_1=1.060416
PR_2=1.061946
PR_3=1.044849
PR_4=0.990530

# Solve for parameters

## State prevalence values

In [52]:
# prevalence values
p_D1 = (PR_1 * p_1 * prevalence_c302) / (PR_1 * prevalence_c302 - prevalence_c302 + 1)
p_D2 = (PR_2 * p_2 * prevalence_c302) / (PR_2 * prevalence_c302 - prevalence_c302 + 1)
p_D3 = (PR_3 * p_3 * prevalence_c302) / (PR_3 * prevalence_c302 - prevalence_c302 + 1)
p_S1 = (-p_1 * prevalence_c302 + p_1) / (PR_1 * prevalence_c302 - prevalence_c302 + 1)
p_S2 = (-p_2 * prevalence_c302 + p_2) / (PR_2 * prevalence_c302 - prevalence_c302 + 1)
p_S3 = (-p_3 * prevalence_c302 + p_3) / (PR_3 * prevalence_c302 - prevalence_c302 + 1)
p_D4 = prevalence_c302 - p_D1 - p_D2 - p_D3
p_S4 = (1 - prevalence_c302) - p_S1 - p_S2 - p_S3

In [53]:
print(p_D1.mean().mean(),
      p_D2.mean().mean(), 
      p_D3.mean().mean(),
      p_D4.mean().mean())
print((p_D1/prevalence_c302).mean().mean(),
      (p_D2/prevalence_c302).mean().mean(), 
      (p_D3/prevalence_c302).mean().mean(),
      (p_D4/prevalence_c302).mean().mean())

0.0006416260716383071 0.002197603163884727 0.005400417726652936 0.01617627610519083
0.02425019697007908 0.0876790180419497 0.22132537048223408 0.6667454145057371


In [54]:
print(p_S1.mean().mean(),
      p_S2.mean().mean(), 
      p_S3.mean().mean(),
      p_S4.mean().mean())
print((p_S1/(1-prevalence_c302)).mean().mean(),
      (p_S2/(1-prevalence_c302)).mean().mean(), 
      (p_S3/(1-prevalence_c302)).mean().mean(),
      (p_S4/(1-prevalence_c302)).mean().mean())

0.022263499323322895 0.08049506743098515 0.206656610434217 0.6661688997441081
0.02286856947658191 0.0825644788359763 0.21182522113935517 0.6827417305480867


In [55]:
for param in [p_D1,p_D2,p_D3,p_D4,p_S1,p_S2,p_S3,p_S4
             ]:
    name = [x for x in globals() if globals()[x] is param][0]
    assert param.min().min()>0, 'negative values'
    assert param.max().max()<1, 'values greater than 1'
    
assert np.all((p_D1+p_D2+p_D3+p_D4+p_S1+p_S2+p_S3+p_S4).round(5)==1), 'Prevalence parameters do not all sum to one'

## Other intermediate variables

We are making that assumption that wasting relative risks apply to 
excess mortality, NOT incidence

In [56]:
m_D1 = (ACMR - csmr_c302 + emr_c302 * (1 - paf_wasting_c302) * RR_1
        - csmr_pem + emr_pem
        - csmr_lri + csmr_lri * (1 - paf_wasting_lri) * RR_lri_1
        - csmr_measles + csmr_measles * (1 - paf_wasting_measles) * RR_measles_1) * p_D1

m_D2 = (ACMR - csmr_c302 + emr_c302 * (1 - paf_wasting_c302) * RR_2
        - csmr_pem + emr_pem
        - csmr_lri + csmr_lri * (1 - paf_wasting_lri) * RR_lri_2
        - csmr_measles + csmr_measles * (1 - paf_wasting_measles) * RR_measles_2) * p_D2

m_D3 = (ACMR - csmr_c302 + emr_c302 * (1 - paf_wasting_c302) * RR_3
        - csmr_pem
        - csmr_lri + csmr_lri * (1 - paf_wasting_lri) * RR_lri_3
        - csmr_measles + csmr_measles * (1 - paf_wasting_measles) * RR_measles_3) * p_D3

print(m_D1.mean().mean(),
     m_D2.mean().mean(),
     m_D3.mean().mean())
print((m_D1/p_D1).mean().mean(),
     (m_D2/p_D2).mean().mean(),
     (m_D3/p_D3).mean().mean())

0.00038471423317196885 0.00035911111452382083 0.0004042774293621333
0.5050310046052648 0.1454853385010369 0.06874839207899716


In [57]:
#di_1 = (incidence_c302 * (1 - diarrhea_paf_incidence) * RR_1_incidence) * p_S1/(1-prevalence_c302)
#di_2 = (incidence_c302 * (1 - diarrhea_paf_incidence) * RR_2_incidence) * p_S2/(1-prevalence_c302)
#di_3 = (incidence_c302 * (1 - diarrhea_paf_incidence) * RR_3_incidence) * p_S3/(1-prevalence_c302)
#di_4 = (incidence_c302 * (1 - diarrhea_paf_incidence)) * p_S4/(1-prevalence_c302)
di_1 = (incidence_c302 * p_S1/(1-prevalence_c302))
di_2 = (incidence_c302 * p_S2/(1-prevalence_c302))
di_3 = (incidence_c302 * p_S3/(1-prevalence_c302))
di_4 = (incidence_c302 * p_S4/(1-prevalence_c302))


print(di_1.mean().mean(), di_2.mean().mean(), di_3.mean().mean(), di_4.mean().mean())
print((di_1/p_S1).mean().mean(), 
      (di_2/p_S2).mean().mean(), 
      (di_3/p_S3).mean().mean(), 
      (di_4/p_S4).mean().mean())

0.051360606032450906 0.17565934019110888 0.4387309086919629 1.4067635449888685
2.1295993365009505 2.1295993365009505 2.1295993365009505 2.1295993365009505


In [58]:
dr_1 = remission_c302 / prevalence_c302 * p_D1
dr_2 = remission_c302 / prevalence_c302 * p_D2
dr_3 = remission_c302 / prevalence_c302 * p_D3
dr_4 = remission_c302 / prevalence_c302 * p_D4

print(dr_1.mean().mean(), dr_2.mean().mean(), dr_3.mean().mean(), dr_4.mean().mean())
print((dr_1/p_D1).mean().mean(), 
      (dr_2/p_D2).mean().mean(), 
      (dr_3/p_D3).mean().mean(), 
      (dr_4/p_D4).mean().mean())

4.088092205404951e-07 1.400193159531524e-06 3.4408523266345556e-06 1.030664294691993e-05
0.0006371455877668046 0.0006371455877668046 0.0006371455877668046 0.0006371455877668046


In [59]:
b_D1 = ACMR * p_D1
b_D2 = ACMR * p_D2
b_D3 = ACMR * p_D3

r_D1tx = t1 * p_D1 
r_D1ux = r2 * p_D1
r_D2 = r3 * p_D2
r_D3 = r4 * p_D3

In [60]:
assert np.all((m_D1 / p_D1) > (m_D2 / p_D2)), 'Mortality rate of D1 state less than D2 state'
assert np.all((m_D2 / p_D2) > (m_D3 / p_D3)), 'Mortality rate of D2 state less than D3 state'

## Incidence rates

In [61]:
i_S1 = b_D1 + di_1 - dr_1 + i1*p_2 - m_D1 - r_D1tx - r_D1ux
i_S2 = b_D1 + b_D2 + 2.0*di_1 - dr_1 - dr_2 + i2*p_3 - m_D1 - m_D2 - r_D1tx - r_D2
i_S3 = b_D1 + b_D2 + b_D3 + 2.0*di_1 + di_3 - dr_1 - dr_2 - dr_3 + i3*p_4 - m_D1 - m_D2 - m_D3 - r_D3
i_D1 = -b_D1 - di_1 + dr_1 + m_D1 + r_D1tx + r_D1ux
i_D2 = -b_D1 - b_D2 - 2.0*di_1 + dr_1 + dr_2 + m_D1 + m_D2 + r_D1tx + r_D2
i_D3 = -b_D1 - b_D2 - b_D3 - 2.0*di_1 - di_3 + dr_1 + dr_2 + dr_3 + m_D1 + m_D2 + m_D3 + r_D3

In [62]:
print(i_S1.mean().mean(), i_S2.mean().mean(), i_S3.mean().mean())
print(i_D1.mean().mean(), i_D2.mean().mean(), i_D3.mean().mean())

0.19475323087643423 0.6022842743898745 2.0542183106959455
-0.04658087168819822 -0.08702469771887152 -0.500545968971103


In [63]:
i_D1/p_D1

draw_0     draw_1    draw_10  \
sex    age_start age_end year_start year_end                                    
Female 0.5       1.0     2019       2020     -72.203728 -72.165332 -72.160803   
       1.0       2.0     2019       2020     -72.464856 -72.627199 -72.739769   
       2.0       5.0     2019       2020     -72.432453 -72.612778 -72.722867   
Male   0.5       1.0     2019       2020     -71.853021 -71.900876 -72.426539   
       1.0       2.0     2019       2020     -72.338606 -72.724969 -72.830804   
       2.0       5.0     2019       2020     -72.275608 -72.685899 -72.804211   

                                               draw_100   draw_101   draw_102  \
sex    age_start age_end year_start year_end                                    
Female 0.5       1.0     2019       2020     -71.702508 -72.220982 -71.991267   
       1.0       2.0     2019       2020     -72.209334 -72.784071 -72.671844   
       2.0       5.0     2019       2020     -72.137244 -72.786177 -72.660809   
Male   0.5       1.0     2019       2020     -72.038369 -72.247924 -72.128402   
       1.0       2.0     2019       2020     -72.589590 -72.947070 -72.856609   
       2.0       5.0     2019       2020     -72.532392 -72.940208 -72.837804   

                                               draw_103   draw_104   draw_105  \
sex    age_start age_end year_start year_end                                    
Female 0.5       1.0     2019       2020     -72.350933 -72.340198 -71.924572   
       1.0       2.0     2019       2020     -72.729805 -72.659935 -72.361908   
       2.0       5.0     2019       2020     -72.714384 -72.654101 -72.328995   
Male   0.5       1.0     2019       2020     -72.398932 -72.441656 -72.311796   
       1.0       2.0     2019       2020     -72.750426 -72.726550 -72.624704   
       2.0       5.0     2019       2020     -72.713010 -72.695150 -72.585504   

                                               draw_106  ...   draw_990  \
sex    age_start age_end year_start year_end             ...              
Female 0.5       1.0     2019       2020     -72.196834  ... -72.407048   
       1.0       2.0     2019       2020     -72.799308  ... -72.828638   
       2.0       5.0     2019       2020     -72.801058  ... -72.822402   
Male   0.5       1.0     2019       2020     -72.141155  ... -72.362151   
       1.0       2.0     2019       2020     -72.746649  ... -72.843533   
       2.0       5.0     2019       2020     -72.700663  ... -72.798152   

                                               draw_991   draw_992   draw_993  \
sex    age_start age_end year_start year_end                                    
Female 0.5       1.0     2019       2020     -72.742552 -72.622098 -72.908987   
       1.0       2.0     2019       2020     -72.815846 -72.973927 -72.971222   
       2.0       5.0     2019       2020     -72.808490 -72.975993 -72.982161   
Male   0.5       1.0     2019       2020     -72.167478 -72.587651 -72.682954   
       1.0       2.0     2019       2020     -72.798436 -72.875425 -72.963274   
       2.0       5.0     2019       2020     -72.764055 -72.869154 -72.959258   

                                               draw_994   draw_995   draw_996  \
sex    age_start age_end year_start year_end                                    
Female 0.5       1.0     2019       2020     -72.722836 -72.841655 -72.821904   
       1.0       2.0     2019       2020     -72.919234 -72.883604 -72.938062   
       2.0       5.0     2019       2020     -72.915042 -72.873139 -72.938401   
Male   0.5       1.0     2019       2020     -72.431072 -72.311275 -72.577502   
       1.0       2.0     2019       2020     -72.825386 -72.798279 -72.846670   
       2.0       5.0     2019       2020     -72.790876 -72.759561 -72.821034   

                                               draw_997   draw_998   draw_999  
sex    age_start age_end year_start year_end                                   
Female 0.5       1.0     2019       2020   

In [64]:
i_S1/p_S1

draw_0    draw_1    draw_10  \
sex    age_start age_end year_start year_end                                  
Female 0.5       1.0     2019       2020      9.623380  9.776435   9.811358   
       1.0       2.0     2019       2020      7.590170  8.081703   7.812634   
       2.0       5.0     2019       2020      7.914067  7.523755   8.664408   
Male   0.5       1.0     2019       2020      9.591205  9.675659  10.327653   
       1.0       2.0     2019       2020      7.854543  8.365822   7.715898   
       2.0       5.0     2019       2020      7.185084  7.746122   7.602710   

                                              draw_100   draw_101  draw_102  \
sex    age_start age_end year_start year_end                                  
Female 0.5       1.0     2019       2020      9.872262   9.801959  9.487620   
       1.0       2.0     2019       2020      8.491733   8.829267  7.765868   
       2.0       5.0     2019       2020      8.890512   8.224651  7.472774   
Male   0.5       1.0     2019       2020      9.808703  10.207641  9.103842   
       1.0       2.0     2019       2020      8.462150   8.170943  8.049691   
       2.0       5.0     2019       2020      7.898052   8.202945  7.706759   

                                              draw_103   draw_104  draw_105  \
sex    age_start age_end year_start year_end                                  
Female 0.5       1.0     2019       2020      9.528718  10.331758  9.724734   
       1.0       2.0     2019       2020      7.051904   7.883583  7.800833   
       2.0       5.0     2019       2020      7.264078   7.605661  7.602506   
Male   0.5       1.0     2019       2020      9.328492   9.315768  9.631631   
       1.0       2.0     2019       2020      7.645622   7.990572  8.166853   
       2.0       5.0     2019       2020      7.519716   7.982502  7.079564   

                                              draw_106  ...   draw_990  \
sex    age_start age_end year_start year_end            ...              
Female 0.5       1.0     2019       2020      9.940574  ...  10.236914   
       1.0       2.0     2019       2020      8.444158  ...   8.318905   
       2.0       5.0     2019       2020      7.403606  ...   7.690630   
Male   0.5       1.0     2019       2020      9.248938  ...   9.845562   
       1.0       2.0     2019       2020      8.353968  ...   7.770097   
       2.0       5.0     2019       2020      8.427265  ...   8.103621   

                                              draw_991   draw_992   draw_993  \
sex    age_start age_end year_start year_end                                   
Female 0.5       1.0     2019       2020      9.635162   9.661880  10.262956   
       1.0       2.0     2019       2020      7.399284   7.651833   8.454742   
       2.0       5.0     2019       2020      7.763704   7.829916   7.617755   
Male   0.5       1.0     2019       2020      9.023899  10.010210  10.058646   
       1.0       2.0     2019       2020      7.417928   8.635246   7.834612   
       2.0       5.0     2019       2020      7.107902   7.246397   8.431879   

                                               draw_994  draw_995   draw_996  \
sex    age_start age_end year_start year_end                                   
Female 0.5       1.0     2019       2020      10.029182  9.028839  10.653212   
       1.0       2.0     2019       2020       7.393978  7.458189   8.447242   
       2.0       5.0     2019       2020       7.282706  8.236371   8.704769   
Male   0.5       1.0     2019       2020       9.479452  9.893128  10.205485   
       1.0       2.0     2019       2020       7.978529  7.383875   8.352114   
       2.0       5.0     2019       2020       7.868563  7.484955   8.086367   

                                              draw_997   draw_998  draw_999  
sex    age_start age_end year_start year_end                                 
Female 0.5       1.0     2019       2020      9.910768  10.099769  9.211788  
       1.0       2.0     2019       2020

In [65]:
RR_i3 = (i_D3 * p_S4) / (i_S3 * p_D4)
RR_i2 = (i_D2 * p_S3) / (i_S2 * p_D3)
RR_i1 = (i_D1 * p_S2) / (i_S1 * p_D2)

In [66]:
print(RR_i3.mean().mean(), RR_i2.mean().mean(), RR_i1.mean().mean())

-9.836818150424483 -4.802733452140232 -8.649175531749409


In [67]:
assert np.all((i1 * p_2).transpose().sort_index().transpose().round(8) == (i_S1 + i_D1).round(8)), 'eq1 untrue'
assert np.all((i2 * p_3).transpose().sort_index().transpose().round(8) == (i_S2 + i_D2).round(8)), 'eq2 untrue'
assert np.all((i3 * p_4).transpose().sort_index().transpose().round(8) == (i_S3 + i_D3).round(8)), 'eq3 untrue'
assert np.all((i_D3 + di_3 + b_D3 + r_D2 + r_D1tx).round(8) 
              == (r_D3 + dr_3 + m_D3 + i_D2).round(8)), 'eq4 untrue'
assert np.all((i_D2 + di_1 + r_D1ux + b_D2).round(8)
             == (r_D2 + dr_2 + i_D1 + m_D2).round(8)), 'eq5 untrue'
assert np.all((i_D1 + di_1 + b_D1).round(8) 
             == (dr_1 + m_D1 + r_D1tx + r_D1ux).round(8)), 'eq6 untrue'

In [68]:
for param in [i_S1, i_S2, i_S3, i_D1, i_D2, i_D3
             ]:
    name = [x for x in globals() if globals()[x] is param][0]
    assert param.min().min()>0, 'negative values'

AssertionError: negative values

In [ ]:
RR_i3.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1)

In [ ]:
RR_i2.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1)

In [ ]:
RR_i1.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1)